In [5]:
import numpy as np
import folium
import folium.plugins as plugins

# Load the XYZ file grid points
data_xyz = np.loadtxt('../resources/Raw/C_SR_resampled.xyz')
lon_xyz = data_xyz[:, 0]
lat_xyz = data_xyz[:, 1]
z_xyz = data_xyz[:, 2]

# Load the points file
data_pts = np.loadtxt('../resources/Raw/pois_eastern_sicily_2km.txt', dtype=str)
id_pts = data_pts[:, 0]
lon_pts = data_pts[:, 1].astype(float)
lat_pts = data_pts[:, 2].astype(float)
z_pts = data_pts[:, 3].astype(float)

# Create a Folium map centered on the data's extent
map = folium.Map(location=[np.mean(lat_xyz), np.mean(lon_xyz)], zoom_start=12)

#Basemaps
folium.TileLayer(tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True,
        show = True).add_to(map)

folium.raster_layers.TileLayer("OpenStreetMap", show=False).add_to(map)
folium.raster_layers.TileLayer("stamentoner", show=False).add_to(map)
folium.raster_layers.TileLayer("stamenterrain", show=False).add_to(map)


# Create a feature group for the bound of Siracusa grid points
fg_gridbound = folium.FeatureGroup(name='Grid Boundary')
xyzbounds = [[lat_xyz.min(), lon_xyz.min()], [lat_xyz.max(), lon_xyz.max()]]
folium.Rectangle(bounds=xyzbounds, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.12).add_to(fg_gridbound)

# Create a feature group for the XYZ grid points
fg_xyz = folium.FeatureGroup(name='Grid Points')

# Add a circle marker for each XYZ data point with a tooltip showing its z value
for lat, lon, z in zip(lat_xyz, lon_xyz, z_xyz):
    folium.CircleMarker(location=[lat, lon],
                        radius=0.1,
                        color='red',
                        opacity=0.4,
                        popup=str(z)).add_to(fg_xyz)

# Create a feature group for the offshore points 
fg_pts = folium.FeatureGroup(name='Points Data')

# Add a marker for each point with a tooltip showing its id
for id, lat, lon,z in zip(id_pts, lat_pts, lon_pts, z_pts):
    info = f"ID:{id}, Depth: {z} "  # Create tooltip string
    folium.Marker(location=[lat, lon],
                  tooltip=info,
                  permanent=True).add_to(fg_pts)



#Read earthquake events data from txt files
data_BS = np.loadtxt('../resources/eve_BS.txt', dtype=str,skiprows=1)
data_PS = np.loadtxt('../resources/eve_PS.txt', dtype=str,skiprows=1)
data_reg = np.loadtxt('../resources/source_region.txt', dtype=str,skiprows=1)

#some filter on events
# data_BS = data_BS[data_BS[:,3].astype(float) >= 9.0]
# data_PS = data_PS[data_PS[:,3].astype(float) >= 9.0]

#Create a feature group for the earthquake events by type
fg_BS = folium.FeatureGroup(name='BS Events')
fg_PS = folium.FeatureGroup(name='PS Events')
fg_reg = folium.FeatureGroup(name='Regions')

#zip inputs of event file and plot as markers
BS_id = data_BS[:,0]
BS_reg = data_BS[:,1]
BS_mag = data_BS[:,3].astype(float)
BS_lon = data_BS[:,4].astype(float)
BS_lat = data_BS[:,5].astype(float)

PS_id = data_PS[:,0]
PS_reg = data_PS[:,1]
PS_mag = data_PS[:,3].astype(float)
PS_lon = data_PS[:,4].astype(float)
PS_lat = data_PS[:,5].astype(float)

reg_id = data_reg[:,0]
reg_type = data_reg[:,1]
reg_UL_lon = data_reg[:,2].astype(float)
reg_UL_lat = data_reg[:,3].astype(float)
reg_LR_lon = data_reg[:,4].astype(float)
reg_LR_lat = data_reg[:,5].astype(float)
reg_bounds = [[reg_UL_lat[i], reg_UL_lon[i]] + [reg_LR_lat[i], reg_LR_lon[i]] for i in range(len(reg_id))]
reg_color = reg_type
reg_color[reg_color == 'BS'] = 'red'
reg_color[reg_color == 'PS'] = 'blue'
reg_color[reg_color == 'BS/PS'] = 'purple'

#Add markers for each event
for id, reg, mag, lon, lat in zip(BS_id, BS_reg, BS_mag, BS_lon, BS_lat):
    info = f"ID:{id}, Region: {reg}, Magnitude: {mag} "  # Create tooltip string
    folium.Marker(location=[lat, lon],
                  tooltip=info,
                  permanent=True,
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(fg_BS)

for id, reg, mag, lon, lat in zip(PS_id, PS_reg, PS_mag, PS_lon, PS_lat):
    info = f"ID:{id}, Region: {reg}, Magnitude: {mag} "  # Create tooltip string
    folium.Marker(location=[lat, lon],
                  tooltip=info,
                  permanent=True,
                  icon=folium.Icon(color='blue', icon='info-sign')).add_to(fg_PS)

#Add rectangles for each region
for id, type, UL_lon, UL_lat, LR_lon, LR_lat, typecolor in zip(reg_id, reg_type, reg_UL_lon, reg_UL_lat, reg_LR_lon, reg_LR_lat, reg_color):
    info = f"ID:{id}, Type: {type} "  # Create tooltip string
    folium.Rectangle(bounds=[[UL_lat, UL_lon], [LR_lat, LR_lon]],
                     color=typecolor,
                     fill=True,
                     fill_color='#ffff00',
                     fill_opacity=0.12,
                     tooltip=info,
                     permanent=True).add_to(fg_reg)
    
#Add the feature groups to the map
fg_gridbound.add_to(map)
fg_xyz.add_to(map)
fg_pts.add_to(map)
fg_BS.add_to(map)
fg_PS.add_to(map)
fg_reg.add_to(map)

# Add a layer control to turn on/off the feature groups
folium.LayerControl().add_to(map)

# Save the map as an HTML file or plot here
map.save('map.html')
# map
